In [12]:
%pip install pip install google-cloud-bigquery

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [13]:
###################################
# Importa bibliotecas necessárias #
###################################

import requests, re , os, time, zipfile, io, boto3, json, pandas as pd
from datetime import datetime , date , timedelta
from google.cloud import bigquery
from google.oauth2 import service_account

In [14]:
##################
# Conecta ao GBQ #
##################

gbq_json_credentials = {} # credenciais ocultadas

gbq_credentials = service_account.Credentials.from_service_account_info(gbq_json_credentials, scopes = ["https://www.googleapis.com/auth/cloud-platform"])

In [15]:
###########################
# Declara a Classe Pandas #
###########################

class Pandas:
    def date_format(date):
        if date:
            return pd.to_datetime(date).strftime("%Y-%m-%dT%H:%M:%S.%Z")

    def data_transform(json , datetime_columns , list_columns):
        df = pd.json_normalize(json)
        if datetime_columns:
             for column in datetime_columns:
                if df[column].isnull().all() == False:
                    df[column] = df[column].astype(str)
                    df[column] = df[column].apply(lambda x : None if x=="NaT" else x)
                    df[column] = df[column].apply(lambda x : None if x=="NaN" else x)
                    df[column] = df[column].apply(lambda x : None if x=="nan" else x)
                    df[column] = df[column].apply(lambda x : None if x=="None" else x)  
                    df[column] = df[column].apply(Pandas.date_format) 
        for column in df.columns:
            df[column] = df[column].apply(lambda x: x if not isinstance(x, list) else " ".join(x))
            
        return df

In [16]:
###########################
# Declara classe BigQuery #
###########################

class BigQuery:
  
    def bq_para_df(project_id, table):
        sql = """
            SELECT MAX(created_at)
            FROM `{project_id}.{table}`
            """.format(project_id=project_id, table=table)
        df = pd.read_gbq(credentials=gbq_credentials, query=sql)
        return df
    
    def data_insertion(df, table , credentials , skip):
        # envia para GBQ
        print("Enviando para GBQ:")
        client = bigquery.Client(credentials=credentials, project=credentials.project_id)
        table_id = "data-lake-364518." + table
        job_config = bigquery.LoadJobConfig()
        job_config.autodetect = True
        job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
        job.result()
        print(f"Dados inseridos na tabela {table} - GBQ - Skip:{skip}")
    

In [17]:
#Pega o dia de ontem
yesterday = date.today() - timedelta(days=1)

In [19]:
datetime_columns = ['storageDate' , 'openDate' , 'closeDate' , 'statusDate' , 'firstResponseDate']
# datetime_columns = []
list_columns = ['tags']

skip = 0
has_tickets = True

blip_chats = [{'name':'transbordoal - Lojas' , 'Authorization': 'Key dHJhbnNib3Jkb2FscHJkOnRrYVE2dG5aeVI5M1ZrVnhZYTI0' , 'table_to':'Data_Lake_Raw.blip_tickets'},
              {'name':'transbordoaf - Entregadores' , 'Authorization': 'Key dHJhbnNib3Jkb3Byb2Q6eVpyZU1KSE1INk9PZ0JIc25wb2c=' , 'table_to':'Data_Lake_Raw.blip_tickets_entregadores'}]

for chat in blip_chats:
  table_to = chat["table_to"]

  # Executa o enquanto houver tickets
  while has_tickets != False:

    url = 'https://americanas.http.msging.net/commands'
    headers = {'Content-Type': 'application/json', 'Authorization': chat["Authorization"]}
    body = {
      "id": "{{$guid}}",
      "to": "postmaster@desk.msging.net",
      "method": "get",
      "uri": f"/tickets?$take=100&$skip={skip}&$filter=(statusDate%20ge%20datetimeoffset'{yesterday}T00:00:00.000Z')"
    }

    response = requests.post( url , headers=headers , json=body)
    if (response.status_code == 200):
        blip_json = response.json()
        blip_tickets = blip_json['resource']['items']
        if blip_tickets:
          has_tickets = True
        else:
          has_tickets = False
          break
        df = Pandas.data_transform(json=blip_tickets , datetime_columns=datetime_columns , list_columns=list_columns)
        try:
          BigQuery.data_insertion(df=df, table=table_to , credentials=gbq_credentials , skip=skip)
        except Exception as e:
          print(e)
        skip += 100
    else: 
      print( 'Erro durante a requisição: ' , response , response.json)
      break

  del table_to , df
  has_tickets = True
  skip = 0

Enviando para GBQ:
Dados inseridos na tabela Data_Lake_Raw.blip_tickets - GBQ - Skip:0
Enviando para GBQ:
Dados inseridos na tabela Data_Lake_Raw.blip_tickets_entregadores - GBQ - Skip:0
Enviando para GBQ:
Dados inseridos na tabela Data_Lake_Raw.blip_tickets_entregadores - GBQ - Skip:100
Enviando para GBQ:
Dados inseridos na tabela Data_Lake_Raw.blip_tickets_entregadores - GBQ - Skip:200
Enviando para GBQ:
Dados inseridos na tabela Data_Lake_Raw.blip_tickets_entregadores - GBQ - Skip:300
Enviando para GBQ:
Dados inseridos na tabela Data_Lake_Raw.blip_tickets_entregadores - GBQ - Skip:400
Enviando para GBQ:
Dados inseridos na tabela Data_Lake_Raw.blip_tickets_entregadores - GBQ - Skip:500
Enviando para GBQ:
Dados inseridos na tabela Data_Lake_Raw.blip_tickets_entregadores - GBQ - Skip:600
